In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(tf.__version__)

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [ ]:
from IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile

In [ ]:
logdir = pathlib.Path(tempfile.mkdtemp())
shutil.rmtree(logdir, ignore_errors=True)

In [ ]:
gz = keras.utils.get_file('HIGGS.csv.gz', 'http://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz')

In [ ]:
FEATURES = 28

In [ ]:
ds = tf.data.experimental.CsvDataset(gz, [float(),] * (FEATURES + 1), compression_type='GZIP')

In [ ]:
print(ds)

In [ ]:
def pack_row(*row):
    label = row[0]
    features = tf.stack(row[1:], 1)

    return features, label

In [ ]:
packed_ds = ds.batch(10000).map(pack_row).unbatch()

In [ ]:
packed_ds

In [ ]:
for features, label in packed_ds.batch(1000).take(1):
    print(features[0])
    
    plt.hist(features.numpy().flatten(), bins = 202)

In [ ]:
N_VALIDATION = 1000
N_TRAIN = 10000
BUFFER_SIZE = 10000
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

In [ ]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

In [ ]:
train_ds

In [ ]:
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BATCH_SIZE).repeat().batch(BATCH_SIZE)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(0.001, STEPS_PER_EPOCH * 1000, 1, False)

def get_optimizer():
    return tf.keras.optimizers.Adam(lr_schedule)

In [ ]:
step = np.linspace(0, 100000)
lr = lr_schedule(step)

plt.figure(figsize = (8, 6))
plt.plot(step / STEPS_PER_EPOCH, lr)
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning rate')

In [ ]:
def get_callbacks():
    return [
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor = 'val_binary_crossentropy', patience = 200),
        tf.keras.callbacks.TensorBoard(logdir/name),
    ]

In [ ]:
def compile_and_fit(model, name, optimizer = None, max_epochs = 10000):
    if optimizer is None:
        optimizer = get_optimizer()
    
    model.compile(
        optimizer = optimizer,
        loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
        metrics = [
            tf.keras.losses.BinaryCrossentropy(from_logits=True, name = 'binary_crossentropy'),
            'accuracy',
        ])
    
    model.summary()

    history = model.fit(train_ds, steps_per_epoch=STEPS_PER_EPOCH, epochs= EPOCHS, validation_data=validate_ds, callbacks=get_callbacks(), verbose=0)

    return history

In [ ]:
tiny_model = tf.keras.models.Sequential([
    layers.Dense(16, activation = 'elu', input_shape = (FEATURES, )),
    layers.Dense(1)
])

In [ ]:
size_histories = {}

In [ ]:
size_histories['Tiny'] = compile_and_fit(tiny_model, 'sizes/Tiny')

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(size_histories)

plt.ylim([0.5, 0.75])

In [ ]:
small_model = tf.keras.models.Sequential([
    layers.Dense(16, 'elu', input_shape = (FEATURES, )),
    layers.Dense(16, 'elu'),
    layers.Dense(1),
])

In [ ]:
size_histories['Small'] = compile_and_fit(small_model, 'sizes/Small')

In [ ]:
medium_model = tf.keras.models.Sequential([
    layers.Dense(64, 'elu', input_shape = (FEATURES, )),
    layers.Dense(64, 'elu'),
    layers.Dense(64, 'elu'),
    layers.Dense(1),
])

In [ ]:
size_histories['Medium'] = compile_and_fit(medium_model, 'sizes/Medium')

In [ ]:
large_model = tf.keras.models.Sequential([
    layers.Dense(512, 'elu', input_shape = (FEATURES, )),
    layers.Dense(512, 'elu'),
    layers.Dense(512, 'elu'),
    layers.Dense(512, 'elu'),
    layers.Dense(1),
])

In [ ]:
size_histories['Large'] = compile_and_fit(large_model, 'sizes/Large')

In [ ]:
plotter.plot(size_histories)
a = plt.xscale('log')

plt.xlim([5, max(plt.xlim())])
plt.ylim([0.5, 0.7])

plt.xlabel('Epoch [Log scale]')

In [ ]:
# shutil.rmtree(logdir/'regularizers/Tiny', ignore_errors=True)
# shutil.copytree(logdir/'sizes/Tiny', logdir/'regularizers/Tiny')

In [ ]:
regularizer_histories = {}
regularizer_histories['Tiny'] = size_histories['Tiny']

In [ ]:
l2_model = tf.keras.models.Sequential([
    layers.Dense(512, 'elu', kernel_regularizer = regularizers.l2(0.001), input_shape = (FEATURES, )),
    layers.Dense(512, 'elu', kernel_regularizer = regularizers.l2(0.001)),
    layers.Dense(512, 'elu', kernel_regularizer = regularizers.l2(0.001)),
    layers.Dense(512, 'elu', kernel_regularizer = regularizers.l2(0.001)),
    layers.Dense(1),
])

In [ ]:
regularizer_histories['l2'] = compile_and_fit(l2_model, 'regularizers/l2')

In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

In [ ]:
dropout_model = tf.keras.models.Sequential([
    layers.Dense(512, 'elu', input_shape = (FEATURES, )),
    layers.Dropout(0.5),
    layers.Dense(512, 'elu'),
    layers.Dropout(0.5),
    layers.Dense(512, 'elu'),
    layers.Dropout(0.5),
    layers.Dense(512, 'elu'),
    layers.Dropout(0.5),
    layers.Dense(1),
])

In [ ]:
regularizer_histories['dropout'] = compile_and_fit(dropout_model, 'regularizers/dropout')

In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

In [ ]:
combined_model = tf.keras.models.Sequential([
    layers.Dense(512, 'elu', kernel_regularizer = regularizers.l2(0.0001), input_shape = (FEATURES, )),
    layers.Dropout(0.5),
    layers.Dense(512, 'elu', kernel_regularizer = regularizers.l2(0.0001)),
    layers.Dropout(0.5),
    layers.Dense(512, 'elu', kernel_regularizer = regularizers.l2(0.0001)),
    layers.Dropout(0.5),
    layers.Dense(512, 'elu', kernel_regularizer = regularizers.l2(0.0001)),
    layers.Dropout(0.5),
    layers.Dense(1),
])

In [ ]:
regularizer_histories['combined'] = compile_and_fit(combined_model, 'regularizers/combined')

In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])